In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers

In [ ]:
class FCLayer(tf.keras.layers.Layer):
    def __init__(self, num_layers, h_dim, dropout_rate=None, activation=tf.nn.relu, kernel_regularizer=None):
        super(FCLayer, self).__init__()
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.fcn = layers.Dense(h_dim, activation=activation,
                                 kernel_initializer='glorot_uniform', kernel_regularizer=kernel_regularizer)
        self.dropout = layers.Dropout(rate=dropout_rate)
        
    def call(self, x):
        for layer in range(self.num_layers):
            x = self.fcn(x)
            if not self.keep_prob is None:
                x = self.dropout(x)
        return x

    

In [ ]:
class DeepHit(Model):
    def __init__(self, num_layers_shared, h_dim_shared, activation, dropout_rate, kernel_regularizer,
                num_layers_CS, h_dim_CS):
        super(DeepHit, self).__init__()
        self.shared_net = FCLayer(num_layers_shared, h_dim_shared, dropout_rate, activation, kernel_regularizer)
        self.cs_net = FCLayer(num_layers_CS, h_dim_CS, dropout_rate, activation, kernel_regularizer)
        
    def call(self, x):
        pass